In [ ]:
import pandas as pd

# Upload your dataset to Colab
from google.colab import files
uploaded = files.upload()

# Load dataset into a DataFrame
df = pd.read_csv("APACHE LOG")

# Inspect data
df.head()
df.info()

Saving apache_logs.csv to apache_logs.csv


FileNotFoundError: [Errno 2] No such file or directory: 'APACHE LOG'

In [ ]:
import pandas as pd

# Use the correct file name
df = pd.read_csv("apache_logs.csv")
df.head()


Timestamp Log_Level Client_IP  \
0  Thu Jun 09 06:07:04 2005    notice         -   
1  Thu Jun 09 06:07:04 2005    notice         -   
2  Thu Jun 09 06:07:04 2005    notice         -   
3  Thu Jun 09 06:07:05 2005    notice         -   
4  Thu Jun 09 06:07:05 2005    notice         -   

                                             Message  
0                 LDAP: Built with OpenLDAP LDAP SDK  
1                      LDAP: SSL support unavailable  
2  suEXEC mechanism enabled (wrapper: /usr/sbin/s...  
3  Digest: generating secret for digest authentic...  
4                                       Digest: done

In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [ ]:
print("First few rows:")
print(df.head())
print("\nData Info:")
print(df.info())

First few rows:
                  Timestamp Log_Level Client_IP  \
0  Thu Jun 09 06:07:04 2005    notice         -   
1  Thu Jun 09 06:07:04 2005    notice         -   
2  Thu Jun 09 06:07:04 2005    notice         -   
3  Thu Jun 09 06:07:05 2005    notice         -   
4  Thu Jun 09 06:07:05 2005    notice         -   

                                             Message  
0                 LDAP: Built with OpenLDAP LDAP SDK  
1                      LDAP: SSL support unavailable  
2  suEXEC mechanism enabled (wrapper: /usr/sbin/s...  
3  Digest: generating secret for digest authentic...  
4                                       Digest: done  

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52004 entries, 0 to 52003
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Timestamp  52004 non-null  object
 1   Log_Level  52004 non-null  object
 2   Client_IP  52004 non-null  object
 3   Message    52004 non-null  

In [ ]:
df.fillna("Unknown", inplace=True)  # Fill missing values

# Convert Timestamp column to datetime (if not already in datetime format)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Extract time features: hour and minute (these may capture diurnal patterns)
df['hour'] = df['Timestamp'].dt.hour
df['minute'] = df['Timestamp'].dt.minute

In [ ]:
# Step 3: Create a Target Label
# We assume logs with "error" (case-insensitive) in Log_Level indicate an issue.
df['issue'] = df['Log_Level'].apply(lambda x: 1 if 'error' in x.lower() else 0)

print("\nLabel distribution:")
print(df['issue'].value_counts())



Label distribution:
issue
1    38081
0    13923
Name: count, dtype: int64


In [ ]:
# Step 4: Feature Selection
# Use the 'Message' text along with the extracted 'hour' and 'minute' features.
X = df[['Message', 'hour', 'minute']]
y = df['issue']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
from xgboost import XGBClassifier
preprocessor = ColumnTransformer(transformers=[
    ('text', TfidfVectorizer(), 'Message'),
    ('num', StandardScaler(), ['hour', 'minute'])
])

# Combine the preprocessor with an XGBoostClassifier into a pipeline.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(n_estimators=100,
                                 use_label_encoder=False,
                                 eval_metric='logloss',
                                 random_state=42))
])


In [ ]:
pipeline.fit(X_train, y_train)

# Step 8: Make Predictions on the Test Set
y_pred = pipeline.predict(X_test)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:26:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
# Step 9: Evaluate the Model
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.64      0.78      4177
           1       0.88      1.00      0.94     11425

    accuracy                           0.90     15602
   macro avg       0.94      0.82      0.86     15602
weighted avg       0.92      0.90      0.90     15602

Accuracy: 0.903858479682092
Confusion Matrix:
[[ 2677  1500]
 [    0 11425]]
